In [88]:
# code to take adoption output and make energy and cost projections

In [89]:
# read in pickle file
import pandas as pd
import numpy as np
baseline_results_df = pd.read_pickle("040_input/baseline_adoption_results.pkl")
tech_results_df = pd.read_pickle("040_input/tech_adoption_results.pkl")
competition_results_df = pd.read_pickle("040_input/competition_adoption_results.pkl")

In [90]:
baseline_results_df

,competition_group,electric_utility,gas_utility,building_type,time,total_stock,ret_er_stock,remaining_stock,rob_stock,remaining_below_baseline_stock,...,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,6000.000000,1800.000000,3600.000000,600.000000,720.000000,...,1080.000000,900.000000,360.000000,450.000000,540.000000,450.000000,120.000000,150.000000,180.000000,150.000000
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,6004.000000,1793.000000,3610.000000,601.000000,599.000000,...,907.833333,752.083333,352.108000,464.720000,532.412000,443.760000,117.404000,158.735000,177.106000,147.755000
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,6008.000000,1790.000000,3619.333333,598.666667,498.336874,...,764.603492,629.027585,326.826513,549.281577,498.814760,415.077149,114.829495,163.487865,174.630253,145.719054
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,6012.000000,1789.555556,3624.777778,597.666667,414.591423,...,645.445302,526.652910,293.587549,667.005010,453.069233,375.893764,106.584395,191.044591,163.673061,136.364620
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,6016.000000,1790.166667,3628.314815,597.518519,344.919946,...,546.312504,441.482998,257.973568,795.484406,403.405756,333.302936,95.744491,229.433407,148.754505,123.586115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,whole_home,test_utility_2,test_utility_2,single_family_li,15,2441.664377,719.756983,1480.815696,241.091699,-2.489802,...,143.823228,18.401161,-2.500000,603.752325,108.859639,9.645020,0.000000,195.296046,41.729966,4.065686
1436,whole_home,test_utility_2,test_utility_2,single_family_li,16,2448.104179,721.688905,1484.679613,241.735661,-2.489828,...,128.698381,18.605244,-2.500000,619.158525,95.200676,9.829704,0.000000,201.184944,36.413098,4.137619
1437,whole_home,test_utility_2,test_utility_2,single_family_li,17,2454.544007,723.620844,1488.543529,242.379635,-2.489855,...,116.146764,18.774786,-2.500000,632.332958,83.804040,9.983845,0.000000,206.211852,31.970049,4.197734
1438,whole_home,test_utility_2,test_utility_2,single_family_li,18,2460.983862,725.552793,1492.407455,243.023615,-2.489881,...,105.730553,18.915663,-2.500000,643.634143,74.306267,10.112383,0.000000,210.512827,28.262874,4.247914


In [91]:
# Process all three dataframes in a loop
dataframes = {
    'baseline': baseline_results_df,
    'tech': tech_results_df,
    'competition': competition_results_df
}

processed_dfs = {}

for name, results_df in dataframes.items():
    print(f"\nProcessing {name}_results_df...")
    
    # Make a copy to avoid modifying original
    df = results_df.copy()
    
    # First step: keep only rows where `time` is an integer (handle floating rounding safely)
    # Ensure `time` is numeric; coerce errors to NaN
    df['time'] = pd.to_numeric(df['time'], errors='coerce')
    
    # Define tolerance for floating rounding when checking integer-ness
    tol = 1e-8
    # Mask rows where time is not null and fractional part is (close to) zero
    mask = df['time'].notna() & np.isclose(df['time'] % 1, 0, atol=tol)
    
    # Keep only those rows and make an explicit copy to avoid SettingWithCopyWarning
    df = df.loc[mask].copy()
    
    # Convert to integer type now that we've filtered
    df['time'] = df['time'].astype(int)
    
    # Quick check
    print(f"Kept {len(df)} rows after filtering to integer times.")
    
    processed_dfs[name] = df

# Display sample of each processed dataframe
for name, df in processed_dfs.items():
    print(f"\n{name} sample:")
    display(df.head())



Processing baseline_results_df...
Kept 1440 rows after filtering to integer times.

Processing tech_results_df...
Kept 1440 rows after filtering to integer times.

Processing competition_results_df...
Kept 1440 rows after filtering to integer times.

baseline sample:


,competition_group,electric_utility,gas_utility,building_type,time,total_stock,ret_er_stock,remaining_stock,rob_stock,remaining_below_baseline_stock,...,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,6000.0,1800.000000,3600.000000,600.000000,720.000000,...,1080.000000,900.000000,360.000000,450.000000,540.000000,450.000000,120.000000,150.000000,180.000000,150.000000
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,6004.0,1793.000000,3610.000000,601.000000,599.000000,...,907.833333,752.083333,352.108000,464.720000,532.412000,443.760000,117.404000,158.735000,177.106000,147.755000
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,6008.0,1790.000000,3619.333333,598.666667,498.336874,...,764.603492,629.027585,326.826513,549.281577,498.814760,415.077149,114.829495,163.487865,174.630253,145.719054
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,6012.0,1789.555556,3624.777778,597.666667,414.591423,...,645.445302,526.652910,293.587549,667.005010,453.069233,375.893764,106.584395,191.044591,163.673061,136.364620
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,6016.0,1790.166667,3628.314815,597.518519,344.919946,...,546.312504,441.482998,257.973568,795.484406,403.405756,333.302936,95.744491,229.433407,148.754505,123.586115



tech sample:


,competition_group,electric_utility,gas_utility,building_type,time,total_stock,ret_er_stock,remaining_stock,rob_stock,remaining_below_baseline_stock,...,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,6000.0,1800.000000,3600.000000,600.000000,720.000000,...,1080.000000,900.000000,360.000000,450.000000,540.000000,450.000000,120.000000,150.000000,180.000000,150.000000
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,6004.0,1793.000000,3610.000000,601.000000,599.000000,...,907.833333,1351.083333,352.108000,440.760000,532.412000,467.720000,117.404000,146.755000,177.106000,159.735000
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,6008.0,1790.000000,3619.333333,598.666667,498.336874,...,764.603492,1727.365382,326.826513,410.120422,498.814760,554.238305,114.829495,143.740694,174.630253,165.466225
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,6012.0,1789.555556,3624.777778,597.666667,414.591423,...,645.445302,2038.088143,293.587549,369.660776,453.069233,673.237998,106.584395,133.748136,163.673061,193.661075
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,6016.0,1790.166667,3628.314815,597.518519,344.919946,...,546.312504,2295.599367,257.973568,326.237515,403.405756,802.549827,95.744491,120.553419,148.754505,232.466103



competition sample:


,competition_group,electric_utility,gas_utility,building_type,time,total_stock,ret_er_stock,remaining_stock,rob_stock,remaining_below_baseline_stock,...,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,6000.0,1800.000000,3600.000000,600.000000,720.000000,...,1080.000000,900.000000,360.000000,450.000000,540.000000,450.000000,120.000000,150.000000,180.000000,150.000000
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,6004.0,1793.000000,3610.000000,601.000000,599.000000,...,1506.833333,752.083333,352.108000,440.760000,556.372000,443.760000,117.404000,146.755000,189.086000,147.755000
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,6008.0,1790.000000,3619.333333,598.666667,498.336874,...,1862.941289,629.027585,326.826513,410.120422,637.975916,415.077149,114.829495,143.740694,194.377424,145.719054
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,6012.0,1789.555556,3624.777778,597.666667,414.591423,...,2156.880535,526.652910,293.587549,369.660776,750.413466,375.893764,106.584395,133.748136,220.969516,136.364620
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,6016.0,1790.166667,3628.314815,597.518519,344.919946,...,2400.428872,441.482998,257.973568,326.237515,872.652648,333.302936,95.744491,120.553419,257.634493,123.586115


In [92]:
# Compute per-step adoptions (differences) on the long-form data for all dataframes
id_cols = ['competition_group', 'electric_utility', 'gas_utility', 'building_type', 'market', 'efficiency_level']

adoption_results = {}

for name, results_df in processed_dfs.items():
    print(f"\nComputing adoptions for {name}_results_df...")
    
    # recreate df_long the same way as before
    import re
    pattern = r'^(ret_er|rob|remaining)_(below_baseline|baseline|efficient|top10)_stock$'
    stock_cols = [c for c in results_df.columns if re.match(pattern, c)]
    df_long = results_df.melt(
        id_vars=['competition_group', 'electric_utility', 'gas_utility', 'building_type', 'time'],
        value_vars=stock_cols,
        var_name='market_eff',
        value_name='value'
    )
    df_long[['market','efficiency_level']] = df_long['market_eff'].str.extract(pattern)
    df_long = df_long.drop(columns=['market_eff'])
    
    # Sort and compute diffs
    df_long = df_long.sort_values(id_cols + ['time'])
    
    grouped = df_long.groupby(id_cols)['value']
    # Backward diff: value - previous value (NaN for first time)
    df_long['adoption'] = grouped.diff().fillna(0)
    # Forward diff: next value - value (NaN for last time)
    df_long['adoption_forward'] = grouped.shift(-1) - df_long['value']
    df_long['adoption_forward'] = df_long['adoption_forward'].fillna(0)
    
    # Pivot adoption (backward diff) to wide
    df_adopt_wide = df_long.pivot_table(
        index=id_cols,
        columns='time',
        values='adoption',
        aggfunc='first'
    ).reset_index()
    # Pivot adoption_forward (change to next step) to wide
    df_adopt_forward_wide = df_long.pivot_table(
        index=id_cols,
        columns='time',
        values='adoption_forward',
        aggfunc='first'
    ).reset_index()
    
    # Rename time columns to adopt_t_<time>
    #import numpy as np
    
    def rename_time_cols(df, prefix='adopt_t'):
        df.columns.name = None
        new_cols = []
        for c in df.columns:
            if isinstance(c, (int, np.integer)):
                new_cols.append(f"{prefix}_{c}")
            else:
                new_cols.append(c)
        df.columns = new_cols
        return df
    
    df_adopt_wide = rename_time_cols(df_adopt_wide, prefix='adopt_t')
    df_adopt_forward_wide = rename_time_cols(df_adopt_forward_wide, prefix='adopt_forward_t')
    
    print(f'Backward-diff adoption shape: {df_adopt_wide.shape}')
    print(f'Forward-diff adoption shape: {df_adopt_forward_wide.shape}')
    
    adoption_results[name] = {
        'adopt_wide': df_adopt_wide,
        'adopt_forward_wide': df_adopt_forward_wide
    }

# Display sample of baseline results
print("\nBaseline adoption sample:")
display(adoption_results['baseline']['adopt_wide'].head())



Computing adoptions for baseline_results_df...
Backward-diff adoption shape: (864, 26)
Forward-diff adoption shape: (864, 26)

Computing adoptions for tech_results_df...
Backward-diff adoption shape: (864, 26)
Forward-diff adoption shape: (864, 26)

Computing adoptions for competition_results_df...
Backward-diff adoption shape: (864, 26)
Forward-diff adoption shape: (864, 26)

Baseline adoption sample:


,competition_group,electric_utility,gas_utility,building_type,market,efficiency_level,adopt_t_0,adopt_t_1,adopt_t_2,adopt_t_3,...,adopt_t_10,adopt_t_11,adopt_t_12,adopt_t_13,adopt_t_14,adopt_t_15,adopt_t_16,adopt_t_17,adopt_t_18,adopt_t_19
0,refrigeration,none,none,multi_family,remaining,baseline,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,refrigeration,none,none,multi_family,remaining,below_baseline,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,refrigeration,none,none,multi_family,remaining,efficient,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,refrigeration,none,none,multi_family,remaining,top10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,refrigeration,none,none,multi_family,ret_er,baseline,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
# Save all processed dataframes to pickle and CSV files
for name, df in processed_dfs.items():
    output_pkl = f"040_output/{name}_formatted_adoption_results.pkl"
    output_csv = f"040_output/{name}_formatted_adoption_results.csv"
    df.to_pickle(output_pkl)
    df.to_csv(output_csv, index=False)
    print(f"Saved {name} results to {output_pkl} and {output_csv}")


Saved baseline results to 040_output/baseline_formatted_adoption_results.pkl and 040_output/baseline_formatted_adoption_results.csv
Saved tech results to 040_output/tech_formatted_adoption_results.pkl and 040_output/tech_formatted_adoption_results.csv
Saved competition results to 040_output/competition_formatted_adoption_results.pkl and 040_output/competition_formatted_adoption_results.csv


In [94]:
# Create difference dataframes: tech vs baseline and competition vs baseline
# The columns ['competition_group', 'electric_utility', 'gas_utility', 'building_type', 'time'] are the merge keys

import re

# Identify the stock columns (columns ending with _stock)
pattern = r'^(ret_er|rob|remaining)_(below_baseline|baseline|efficient|top10)_stock$'
baseline_df = processed_dfs['baseline']
stock_cols = [c for c in baseline_df.columns if re.match(pattern, c)]

# Merge keys
merge_keys = ['competition_group', 'electric_utility', 'gas_utility', 'building_type', 'time']

# Initialize dictionary to store difference dataframes
diff_dfs = {}

for scenario in ['tech', 'competition']:
    print(f"\nComputing differences: {scenario} - baseline...")
    
    scenario_df = processed_dfs[scenario]
    
    # Merge baseline and scenario dataframes on the key columns
    merged = baseline_df[merge_keys + stock_cols].merge(
        scenario_df[merge_keys + stock_cols],
        on=merge_keys,
        suffixes=('_baseline', f'_{scenario}')
    )
    
    # Create the difference dataframe starting with merge keys
    diff_df = merged[merge_keys].copy()
    
    # Compute differences for each stock column: scenario - baseline
    for col in stock_cols:
        baseline_col = f"{col}_baseline"
        scenario_col = f"{col}_{scenario}"
        diff_df[col] = merged[scenario_col] - merged[baseline_col]
    
    diff_dfs[scenario] = diff_df
    print(f"Shape of {scenario} difference dataframe: {diff_df.shape}")
    print(f"Sample:")
    display(diff_df.head())

print("\nDifference dataframes created successfully!")



Computing differences: tech - baseline...
Shape of tech difference dataframe: (1440, 17)
Sample:


,competition_group,electric_utility,gas_utility,building_type,time,remaining_below_baseline_stock,remaining_baseline_stock,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,0.0,-599.000000,0.0,599.000000,0.0,-23.960000,0.0,23.960000,0.0,-11.980000,0.0,11.980000
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,0.0,-1098.337797,0.0,1098.337797,0.0,-139.161156,0.0,139.161156,0.0,-19.747171,0.0,19.747171
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,0.0,-1511.435233,0.0,1511.435233,0.0,-297.344234,0.0,297.344234,0.0,-57.296455,0.0,57.296455
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,0.0,-1854.116368,0.0,1854.116368,0.0,-469.246891,0.0,469.246891,0.0,-108.879988,0.0,108.879988



Computing differences: competition - baseline...
Shape of competition difference dataframe: (1440, 17)
Sample:


,competition_group,electric_utility,gas_utility,building_type,time,remaining_below_baseline_stock,remaining_baseline_stock,remaining_efficient_stock,remaining_top10_stock,ret_er_below_baseline_stock,ret_er_baseline_stock,ret_er_efficient_stock,ret_er_top10_stock,rob_below_baseline_stock,rob_baseline_stock,rob_efficient_stock,rob_top10_stock
0,refrigeration,test_utility_1,test_utility_1,multi_family,0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,refrigeration,test_utility_1,test_utility_1,multi_family,1,0.0,-599.000000,599.000000,0.0,0.0,-23.960000,23.960000,0.0,0.0,-11.980000,11.980000,0.0
2,refrigeration,test_utility_1,test_utility_1,multi_family,2,0.0,-1098.337797,1098.337797,0.0,0.0,-139.161156,139.161156,0.0,0.0,-19.747171,19.747171,0.0
3,refrigeration,test_utility_1,test_utility_1,multi_family,3,0.0,-1511.435233,1511.435233,0.0,0.0,-297.344234,297.344234,0.0,0.0,-57.296455,57.296455,0.0
4,refrigeration,test_utility_1,test_utility_1,multi_family,4,0.0,-1854.116368,1854.116368,0.0,0.0,-469.246891,469.246891,0.0,0.0,-108.879988,108.879988,0.0



Difference dataframes created successfully!


In [95]:
# Save all processed dataframes to pickle and CSV files
for name, df in diff_dfs.items():
    output_pkl = f"040_output/{name}_diff_adoption_results.pkl"
    output_csv = f"040_output/{name}_diff_adoption_results.csv"
    df.to_pickle(output_pkl)
    df.to_csv(output_csv, index=False)
    print(f"Saved {name} results to {output_pkl} and {output_csv}")


Saved tech results to 040_output/tech_diff_adoption_results.pkl and 040_output/tech_diff_adoption_results.csv
Saved competition results to 040_output/competition_diff_adoption_results.pkl and 040_output/competition_diff_adoption_results.csv


In [96]:
# write code to take all of the outputted csvs in this notebook and put them into a single excel workbook with each dataframe as its own sheet

with pd.ExcelWriter("040_output/adoption_results_summary.xlsx") as writer:
    # Write processed dataframes
    for name, df in processed_dfs.items():
        sheet_name = f"{name}_formatted"
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Wrote {sheet_name} to Excel.")
    
    # Write difference dataframes
    for name, df in diff_dfs.items():
        sheet_name = f"{name}_diff"
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Wrote {sheet_name} to Excel.")
    
    # Write adoption results from Cell 5
    for name, results in adoption_results.items():
        # Write backward diff adoption
        sheet_name = f"{name}_adopt_wide"
        results['adopt_wide'].to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Wrote {sheet_name} to Excel.")
        
        # Write forward diff adoption
        sheet_name = f"{name}_adopt_forward"
        results['adopt_forward_wide'].to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Wrote {sheet_name} to Excel.")


Wrote baseline_formatted to Excel.
Wrote tech_formatted to Excel.
Wrote competition_formatted to Excel.
Wrote tech_diff to Excel.
Wrote competition_diff to Excel.
Wrote baseline_adopt_wide to Excel.
Wrote baseline_adopt_forward to Excel.
Wrote tech_adopt_wide to Excel.
Wrote tech_adopt_forward to Excel.
Wrote competition_adopt_wide to Excel.
Wrote competition_adopt_forward to Excel.
